# Werkcollege-opdrachten Week 1.2

## Voorbereiding

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [1]:

import warnings
warnings.simplefilter('ignore')

import pandas as pd
import sqlite3

Zet de volgende bestanden in een makkelijk terug te vinden map:
- go_sales_train.sqlite
- go_crm_train.sqlite
- go_staff_train.sqlite

Bestudeer de bovenste 3 bestanden in DB Browser (SQLite), <a href="https://sqlitebrowser.org/dl/">hier</a> te downloaden. Wat valt je op qua datatypen?<br>

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales_train.sqlite.

In [2]:
db_path = 'Sqlite files/go_sales_train.sqlite'

sales_conn = sqlite3.connect(db_path)
sales_conn

<b>Let goed op:</b><br>
Als je per ongeluk een verkeerde bestandsnaam ingeeft, maakt Python zélf een leeg databasebestand aan! Er ontstaat dan dus een nieuwe .sqlite, en dat is nadrukkelijk <u>niet de bedoeling.</u>

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales_train zitten.

In [3]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in
tables = pd.read_sql(sql_query, sales_conn)
print(tables)

             name
0         country
1   order_details
2    order_header
3    order_method
4         product
5    product_line
6    product_type
7   retailer_site
8   return_reason
9   returned_item
10   sales_branch
11    sales_staff


Krijg je lege output? Dan heb je per ongeluk een leeg  databasebestand (.sqlite) aangemaakt.<br>
Lees de informatie onder het kopje <u>Let goed op:</u> hierboven nog eens goed door.

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [5]:
sales_staff = pd.read_sql("SELECT * FROM sales_staff", sales_conn)
sales_staff

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


## Datumkolommen

Zoals je misschien al hebt gezien in DB Browser, zijn datums als TEXT opgeslagen, en niet als DATE, DATETIME o.i.d. Hier moeten we dus nog even "typische datumkolommen" van maken. Dat doen we met de volgende code:

In [7]:
sales_staff['DATE_HIRED'] = pd.to_datetime(sales_staff['DATE_HIRED'])
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
dtype: object

Als we hier het jaar uit willen halen, schrijven we:

In [8]:
pd.DatetimeIndex(sales_staff['DATE_HIRED']).quarter

Index([4, 2, 2, 3, 1, 2, 1, 1, 3, 3,
       ...
       3, 1, 4, 1, 3, 4, 4, 4, 4, 1],
      dtype='int32', name='DATE_HIRED', length=102)

Deze zelfde syntax is te gebruiken voor het extraheren van kwartalen, maanden, weken en dagen. Probeer het maar eens!

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [45]:
sales_staff.loc[:,  
                ['SALES_STAFF_CODE', 'WORK_PHONE',
                  'EXTENSION', 'FAX', 'EMAIL']]


,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com
...,...,...,...,...,...
97,120,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com
98,121,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com
99,122,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com
100,123,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com


Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [46]:
contact_details = pd.concat([sales_staff['SALES_STAFF_CODE'], 
                             sales_staff['WORK_PHONE'], 
                             sales_staff['EXTENSION'], 
                             sales_staff['FAX'], 
                             sales_staff['EMAIL']], axis= 1)
contact_details

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com
...,...,...,...,...,...
97,120,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com
98,121,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com
99,122,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com
100,123,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com


## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [47]:
contact_details.head(5)

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com


Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [20]:
eerste_taal = ['EN'] * 5

taalseries = pd.Series(eerste_taal, name='FIRST_LANGUAGE')

ENsprekers_lijst = pd.concat([taalseries, 
                             sales_staff['SECOND_LANGUAGE'], 
                             sales_staff['THIRD_LANGUAGE']], axis= 1)

ENsprekers_lijst.head(5)

KeyError: 'SECOND_LANGUAGE'

Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [ ]:
mijn_dict = {'EN' }

Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [48]:
data = ['S101', 'John', 'Doe', 'representative', '123456789', 123, '123456789', 'voorbeeld@example.com', '2023-01-01', 1, 'John Doe']
sales_staff.loc[len(sales_staff)] = data
sales_staff

ValueError: cannot set a row with mismatched columns

### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

In [ ]:
sales_staff['FULL_NAME'] = sales_staff['FIRST_NAME'] + ' ' + sales_staff['LAST_NAME']
sales_staff

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FULL_NAME
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03 00:00:00,6,Denis Pagé
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08 00:00:00,6,Élizabeth Michel
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07 00:00:00,6,Émile Clermont
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16 00:00:00,6,Étienne Jauvin
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27 00:00:00,13,Elsbeth Wiesinger
...,...,...,...,...,...,...,...,...,...,...,...
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01 00:00:00,38,François De Crée
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09 00:00:00,38,Yvette Lattrez
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28 00:00:00,39,Willi Seefelder
101,124,Sabine,Grüner,Level 3 Sales Representative,+(43) 13 79 56 32,348.0,+(43) 13 79 56 33,SGruner@grtd123.com,1998-02-18 00:00:00,39,Sabine Grüner


## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

In [ ]:
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
dtype: object

Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(int)
sales_staff.dtypes

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(float)
sales_staff.dtypes

SALES_STAFF_CODE      object
FIRST_NAME            object
LAST_NAME             object
POSITION_EN           object
WORK_PHONE            object
EXTENSION            float64
FAX                   object
EMAIL                 object
DATE_HIRED            object
SALES_BRANCH_CODE      int64
FULL_NAME             object
dtype: object

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'] + 1
sales_staff

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,326.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,337.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,379.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,399.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1819.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1341.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1643.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1634.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,326.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

In [ ]:
sales_staff.loc[sales_staff['POSITION_EN'] == 'Branch Manager', 'POSITION_EN'] = 'General Manager'
sales_staff

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,326.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,337.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,379.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,399.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1819.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1341.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1643.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1634.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,326.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

In [52]:
sales_staff.rename(columns = {'POSITION_EN': 'POSITION'}, inplace = True)
sales_staff

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

In [ ]:
sales_staff.drop([sales_staff.index[99], sales_staff.index[100], sales_staff.index[101]], inplace = True)
sales_staff


,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
94,117,Frank,Jever,Level 3 Sales Representative,+(41) 17 12 13 14,1154.0,+(41) 17 12 13 20,FJever@grtd123.com,1999-10-18,37
95,118,Gianni,Vertemati,Level 1 Sales Representative,+(41) 17 12 13 14,1254.0,+(41) 17 12 13 20,GVertemati@grtd123.com,2000-02-10,37
96,119,Gracy,Gellens,Branch Manager,+32 16 20.73.21,1349.0,+32 16 20.73.32,GGellens@grtd123.com,1999-09-10,38
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38


### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.

In [ ]:
#sales_staff.drop(columns = ['FAX'], axis= 1, inplace = True)
sales_staff.columns
# de FAX kolom zit er niet in, maar de kolom was nog niet verwijderd

Index(['SALES_STAFF_CODE', 'FIRST_NAME', 'LAST_NAME', 'POSITION', 'WORK_PHONE',
       'EXTENSION', 'EMAIL', 'DATE_HIRED', 'SALES_BRANCH_CODE'],
      dtype='object')